In [2]:
import pyrootutils
root = pyrootutils.setup_root(".", pythonpath=True, cwd=True)

In [3]:
import pandas as pd
import numpy as np
import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image, ImageOps

from src.data.face_age_dataset import FaceAgeDataset
from src.data.face_age_datamodule import FaceAgeDataModule

In [4]:
ds = FaceAgeDataset()

In [6]:
x, y = ds[0]
x.shape, y.shape

(torch.Size([3, 224, 224]), torch.Size([1]))